In [ ]:
import random, re, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow as tf, tensorflow.keras.backend as K
from kaggle_datasets import KaggleDatasets
print('Tensorflow version ' + tf.__version__)
from sklearn.model_selection import KFold

# Configurations

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
IMAGE_SIZE = [512, 512]
EPOCHS = 25
FOLDS = 2
# SEED = 777
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:
MIXED_PRECISION = False
XLA_ACCELERATE = False

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

# Data Directories

In [ ]:
# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path('flower-classification-with-tpus')

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}

GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tf-flower-photo-tfrec')
# GCS_DS_PATH = glob.glob(GCS_DS_PATH + '/*')

GCS_PATH_SELECT = { # available image sizes
    512: GCS_DS_PATH + '/*/tfrecords-jpeg-512x512'
}

GCS_PATH1 = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec') + tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec') + tf.io.gfile.glob(GCS_PATH1 + '/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') # predictions on this dataset should be submitted for the competition

# Classes

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

# Custom LR scheduler
From starter [kernel][1]

[1]: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

In [ ]:
# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.

LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

# Dataset Functions
From starter [kernel][1]

[1]: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3])
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3])
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    return image, label   

def get_training_dataset(dataset,do_aug=True):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    if do_aug: dataset = dataset.map(transform, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(dataset):
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = int( count_data_items(TRAINING_FILENAMES))
# NUM_VALIDATION_IMAGES = int( count_data_items(TRAINING_FILENAMES) )
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES,NUM_TEST_IMAGES))

In [ ]:
dataset = load_dataset(TRAINING_FILENAMES, labeled=True)

In [ ]:
# def decode_image(image_data):
#     image = tf.image.decode_jpeg(image_data, channels=3)
#     image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
#     image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
#     return image

# def read_labeled_tfrecord(example):
#     LABELED_TFREC_FORMAT = {
#         "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
#         "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
#     }
#     example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
#     image = decode_image(example['image'])
#     label = tf.cast(example['class'], tf.int32)
#     return image, label # returns a dataset of (image, label) pairs

# def read_unlabeled_tfrecord(example):
#     UNLABELED_TFREC_FORMAT = {
#         "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
#         "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
#         # class is missing, this competitions's challenge is to predict flower classes for the test dataset
#     }
#     example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
#     image = decode_image(example['image'])
#     idnum = example['id']
#     return image, idnum # returns a dataset of image(s)

# def load_dataset(filenames, labeled = True, ordered = False):
#     # Read from TFRecords. For optimal performance, reading from multiple files at once and
#     # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
#     ignore_order = tf.data.Options()
#     if not ordered:
#         ignore_order.experimental_deterministic = False # disable order, increase speed
        
#     dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO) # automatically interleaves reads from multiple files
#     dataset = dataset.with_options(ignore_order) # use data as soon as it streams in, rather than in its original order
#     dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
#     return dataset

# def data_augment(image, label):
#     # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
#     # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
#     # of the TPU while the TPU itself is computing gradients.
#     image = tf.image.random_flip_left_right(image)
#     return image, label   

# def get_training_dataset(dataset,do_aug=True):
#     dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
#     dataset = dataset.map(transform, num_parallel_calls=AUTO)
#     dataset = dataset.repeat() # the training dataset must repeat for several epochs
#     dataset = dataset.shuffle(2048)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def get_validation_dataset(dataset):
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.cache()
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def get_test_dataset(ordered=False):
#     dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def count_data_items(filenames):
#     # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
#     n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
#     return np.sum(n)

# NUM_TRAINING_IMAGES = int( count_data_items(TRAINING_FILENAMES))
# # NUM_VALIDATION_IMAGES = int( count_data_items(TRAINING_FILENAMES) * (1./FOLDS) )
# NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

# print('Dataset: {} training images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))

# Display Example Augmentation
Below are examples of 3 training images where each is randomly augmented 12 different times.

In [ ]:
# row = 3; col = 4;
# all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
# one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)))
# augmented_element = one_element.repeat().map(transform).batch(row*col)

# for (img,label) in augmented_element:
#     plt.figure(figsize=(15,int(15*row/col)))
#     for j in range(row*col):
#         plt.subplot(row,col,j+1)
#         plt.axis('off')
#         plt.imshow(img[j,])
#     plt.show()
#     break

In [ ]:
# row = 3; col = 4;
# all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
# one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)))
# augmented_element = one_element.repeat().map(transform).batch(row*col)

# for (img,label) in augmented_element:
#     plt.figure(figsize=(15,int(15*row/col)))
#     for j in range(row*col):
#         plt.subplot(row,col,j+1)
#         plt.axis('off')
#         plt.imshow(img[j,])
#     plt.show()
#     break

In [ ]:
# row = 3; col = 4;
# all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug=False).unbatch()
# one_element = tf.data.Dataset.from_tensors( next(iter(all_elements)) )
# augmented_element = one_element.repeat().map(transform).batch(row*col)

# for (img,label) in augmented_element:
#     plt.figure(figsize=(15,int(15*row/col)))
#     for j in range(row*col):
#         plt.subplot(row,col,j+1)
#         plt.axis('off')
#         plt.imshow(img[j,])
#     plt.show()
#     break

# Build, Train, Infer Model
This is the 5-Fold workflow copied from Ragnar's notebook [here][1]. Now we add data augmentation to the training images on the fly! Notice how his notebook completes epochs in 70 seconds using TPU. This notebook also completes epochs in 70 seconds (when we turn on TPU) and we are augmentating every image! Augmenting a single image requires 5,000,000 calculations (a batch requires 600,000,000 calculations!) We see that our augmentation is occuring as fast as the GPU/TPU training! We are augmenting 200+ images per second. In other words we are performing 1,000,000,000 calculations per second in addition to normal training computation! Wow!

[1]: https://www.kaggle.com/ragnar123/5-kfold-densenet201

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn
from tensorflow.keras.applications import DenseNet201

In [ ]:
# # def get_model():
# #     with strategy.scope():
# #         rnet = efn.EfficientNetB7(
# #             input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
# #             weights='imagenet',
# #             include_top=False
# #         )
# #         # trainable rnet
# #         rnet.trainable = True
# #         model = tf.keras.Sequential([
# #             rnet,
# #             tf.keras.layers.GlobalAveragePooling2D(),
# #             tf.keras.layers.Dense(len(CLASSES), activation='softmax',dtype='float32')
# #         ])
# #     model.compile(
# #         optimizer='adam',
# #         loss = 'sparse_categorical_crossentropy',
# #         metrics=['sparse_categorical_accuracy']
# #     )
# #     return model


# def get_model():
#     with strategy.scope():
#         rnet = DenseNet201(
#             input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
#             weights='imagenet',
#             include_top=False
#         )
#         # trainable rnet
#         rnet.trainable = True
#         model = tf.keras.Sequential([
#             rnet,
#             tf.keras.layers.GlobalAveragePooling2D(),
#             tf.keras.layers.Dense(len(CLASSES), activation='softmax',dtype='float32')
#         ])
#     model.compile(
#         optimizer='adam',
#         loss = 'sparse_categorical_crossentropy',
#         metrics=['sparse_categorical_accuracy']
#     )
#     return model


# model = get_model()

# def train_cross_validate(folds = 2):
#     histories = []
#     models = []
#     early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
#     kfold = KFold(folds, shuffle = True, random_state = SEED)
#     for f, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
#         print(); print('#'*25)
#         print('### FOLD',f+1)
#         print('#'*25)
#         train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
# #         val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
#         model = get_model()
#         history = model.fit(
#             get_training_dataset(train_dataset),
#             steps_per_epoch = STEPS_PER_EPOCH,
#             epochs = EPOCHS,
#             callbacks = [lr_callback, early_stopping],
#         )
#         models.append(model)
#         histories.append(history)
#     return histories, models

# def train_and_predict(folds = 2):
#     test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.
#     test_images_ds = test_ds.map(lambda image, idnum: image)
#     print('Start training %i folds'%folds)
#     histories, models = train_cross_validate(folds = folds)
#     print('Computing predictions...')
#     # get the mean probability of the folds models
#     probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
#     predictions = np.argmax(probabilities, axis=-1)
#     print('Generating submission.csv file...')
#     test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
#     test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
#     np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
#     return histories, models
    
# # run train and predict
# histories, models = train_and_predict(folds = FOLDS)

In [ ]:
# from tensorflow.keras.applications import DenseNet201
# import efficientnet.tfkeras as efn

# def get_model():
#     with strategy.scope():
#         rnet = efn.EfficientNetB7(
#             input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
#             weights='noisy-student',
#             include_top=False
#         )
#         # trainable rnet
#         rnet.trainable = True
#         model = tf.keras.Sequential([
#             rnet,
#             tf.keras.layers.GlobalAveragePooling2D(),
#             tf.keras.layers.Dense(len(CLASSES), activation='softmax',dtype='float32')
#         ])
#     model.compile(
#         optimizer='adam',
#         loss = 'sparse_categorical_crossentropy',
#         metrics=['sparse_categorical_accuracy']
#     )
#     return model

# def train_cross_validate(folds = 5):
#     histories = []
#     models = []
#     early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
#     kfold = KFold(folds, shuffle = True, random_state = SEED)
#     for f, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
#         print(); print('#'*25)
#         print('### FOLD',f+1)
#         print('#'*25)
#         train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
#         val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
#         model = get_model()
#         history = model.fit(
#             get_training_dataset(train_dataset), 
#             steps_per_epoch = STEPS_PER_EPOCH,
#             epochs = EPOCHS,
#             callbacks = [lr_callback],#, early_stopping],
#             validation_data = get_validation_dataset(val_dataset),
#             verbose=2
#         )
#         models.append(model)
#         histories.append(history)
#     return histories, models

# def train_and_predict(folds = 5):
#     test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.
#     test_images_ds = test_ds.map(lambda image, idnum: image)
#     print('Start training %i folds'%folds)
#     histories, models = train_cross_validate(folds = folds)
#     print('Computing predictions...')
#     # get the mean probability of the folds models
#     probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
#     predictions = np.argmax(probabilities, axis=-1)
#     print('Generating submission.csv file...')
#     test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
#     test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
#     np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
#     return histories, models
    
# # run train and predict
# histories, models = train_and_predict(folds = FOLDS)

# Confusion Matrix and Validation Score
Try forking and modifying this notebook to maximize validation score below. Tune the data augmentation and/or train for more epochs to increase accuracy. Good luck! (Code below is from starter [kernel][1]).

[1]: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu

In [ ]:
# def display_confusion_matrix(cmat, score, precision, recall):
#     plt.figure(figsize=(15,15))
#     ax = plt.gca()
#     ax.matshow(cmat, cmap='Reds')
#     ax.set_xticks(range(len(CLASSES)))
#     ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
#     plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
#     ax.set_yticks(range(len(CLASSES)))
#     ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
#     plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
#     titlestring = ""
#     if score is not None:
#         titlestring += 'f1 = {:.3f} '.format(score)
#     if precision is not None:
#         titlestring += '\nprecision = {:.3f} '.format(precision)
#     if recall is not None:
#         titlestring += '\nrecall = {:.3f} '.format(recall)
#     if len(titlestring) > 0:
#         ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
#     plt.show()

In [ ]:
# %%time
# all_labels = []; all_prob = []; all_pred = []
# kfold = KFold(FOLDS, shuffle = True, random_state = SEED)
# for j, (trn_ind, val_ind) in enumerate( kfold.split(TRAINING_FILENAMES) ):
#     print('Inferring fold',j+1,'validation images...')
#     VAL_FILES = list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES'])
#     NUM_VALIDATION_IMAGES = count_data_items(VAL_FILES)
#     cmdataset = get_validation_dataset(load_dataset(VAL_FILES, labeled = True, ordered = True))
#     images_ds = cmdataset.map(lambda image, label: image)
#     labels_ds = cmdataset.map(lambda image, label: label).unbatch()
#     all_labels.append( next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() ) # get everything as one batch
#     prob = models[j].predict(images_ds)
#     all_prob.append( prob )
#     all_pred.append( np.argmax(prob, axis=-1) )
# cm_correct_labels = np.concatenate(all_labels)
# cm_probabilities = np.concatenate(all_prob)
# cm_predictions = np.concatenate(all_pred)

In [ ]:
# print("Correct   labels: ", cm_correct_labels.shape, cm_correct_labels)
# print("Predicted labels: ", cm_predictions.shape, cm_predictions)

In [ ]:
# cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
# score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
# display_confusion_matrix(cmat, score, precision, recall)
# print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

Efficientnet + Densenet

In [ ]:
LR_START = 0.0001
LR_MAX = 0.000000015 * strategy.num_replicas_in_sync
LR_MIN = 0.00000005
LR_RAMPUP_EPOCHS = 4
LR_SUSTAIN_EPOCHS = 6
LR_EXP_DECAY = .8

def lrfn(epoch):
#     if epoch < LR_RAMPUP_EPOCHS:
#         lr = np.random.random_sample() * LR_START
#     elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
#         lr = LR_MAX
#     else:
    lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch) + LR_MIN
#     lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN

    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

es_callback = tf.keras.callbacks.EarlyStopping(min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
# with strategy.scope():
#     dn201 = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
#     dn201.trainable = True # Full Training
    
#     model1 = tf.keras.Sequential([
#         dn201,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])
        
# model1.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy']
# )
# model1.summary()

In [ ]:
# with strategy.scope():
#     enb7 = efn.EfficientNetB7(weights='noisy-student', include_top=False, input_shape=[*IMAGE_SIZE, 3])
#     enb7.trainable = True # Full Training
    
#     model2 = tf.keras.Sequential([
#         enb7,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ])

# model2.load_weights('/kaggle/input/flowermodel/flower_efficientnet_final1.h5')
    
# model2.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False),
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy']
# )
# model2.summary()

In [ ]:
# history1 = model1.fit(get_training_dataset(dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=30, callbacks = [lr_callback,es_callback])

In [ ]:
# model1.save('flower_densenet.h5')

In [ ]:
# history2 = model2.fit(get_training_dataset(dataset), steps_per_epoch=STEPS_PER_EPOCH, epochs=6, callbacks = [lr_callback,es_callback])

In [ ]:
# model2.save('flower_efficientnet_final1.h5')

In [ ]:
model1 = tf.keras.models.load_model('/kaggle/input/flowermodel/flower_densenet_final1.h5')
model2 = tf.keras.models.load_model('/kaggle/input/flowermodel/flower_efficientnet_final1.h5')

In [ ]:
best_alpha = 0.7

In [ ]:
test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
#probabilities = (model1.predict(test_images_ds)+model2.predict(test_images_ds))/2 # using average model probability
probabilities = model1.predict(test_images_ds)*(1-best_alpha) + model2.predict(test_images_ds)*best_alpha
# probabilities = model2.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
!head submission.csv